In [1]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd
import os
import datetime
import time
import folium
from shapely.geometry import LineString

In [322]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [323]:
mrt_file_loc = "../datasets/sg_mrt_lrt/mrt_stations.shp"
mrt_gdf = gpd.read_file(mrt_file_loc)

mrt_gdf["OPENING"] = pd.to_datetime(mrt_gdf["OPENING"])
mrt_gdf = mrt_gdf.rename(columns={"STATION_NA": "STATION_NAME"})
mrt_gdf["lat_lng"] = gpd.points_from_xy(mrt_gdf["LATITUDE"], mrt_gdf["LONGITUDE"])

mrt_gdf.head()
# mrt_gdf.crs

,CODE,STATION_NAME,LINE,COLOR,OPENING,TYPE,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,geometry,lat_lng
0,NS1,Jurong East,North-South Line,Red,1990-03-10,MRT,JURONG EAST MRT STATION (EW24 / NS1),10,JURONG EAST STREET 12,JURONG EAST MRT STATION (EW24 / NS1),10 JURONG EAST STREET 12 JURONG EAST MRT STATI...,609690,17869.0570516568,35038.9688695427,1.33315281585758,103.742286332403,POINT (17869.057 35038.969),POINT (1.333 103.742)
1,NS2,Bukit Batok,North-South Line,Red,1990-03-10,MRT,BUKIT BATOK MRT STATION (NS2),10,BUKIT BATOK CENTRAL,BUKIT BATOK MRT STATION (NS2),10 BUKIT BATOK CENTRAL BUKIT BATOK MRT STATION...,659958,18679.3223191258,36794.9260214306,1.34903331201636,103.749566478309,POINT (18679.322 36794.926),POINT (1.349 103.750)
2,NS3,Bukit Gombak,North-South Line,Red,1990-03-10,MRT,BUKIT GOMBAK MRT STATION (NS3),802,BUKIT BATOK WEST AVENUE 5,BUKIT GOMBAK MRT STATION (NS3),802 BUKIT BATOK WEST AVENUE 5 BUKIT GOMBAK MRT...,659083,18926.9127006577,37854.0358848721,1.35861159094192,103.751790910733,POINT (18926.913 37854.036),POINT (1.359 103.752)
3,NS4,Choa Chu Kang,North-South Line,Red,1990-03-10,MRT,CHOA CHU KANG MRT STATION (NS4),10,CHOA CHU KANG AVENUE 4,CHOA CHU KANG MRT STATION (NS4),10 CHOA CHU KANG AVENUE 4 CHOA CHU KANG MRT ST...,689810,18101.2472041463,40812.1216052417,1.38536316540225,103.744370779756,POINT (18101.247 40812.122),POINT (1.385 103.744)
4,NS5,Yew Tee,North-South Line,Red,1996-02-10,MRT,YEW TEE MRT STATION (NS5),61,CHOA CHU KANG DRIVE,YEW TEE MRT STATION (NS5),61 CHOA CHU KANG DRIVE YEW TEE MRT STATION (NS...,689715,18438.9831173802,42158.0181021616,1.39753506936297,103.747405150236,POINT (18438.983 42158.018),POINT (1.398 103.747)


In [324]:
def check_codes(row):
    for_check = row["CODE"]
    if len(for_check) == 3 and for_check[-1].isdigit():
        result = for_check.replace(for_check[-1], f"0{for_check[-1]}")
        return result
    else:
        result = for_check
        return result


mrt_gdf["CODE"] = mrt_gdf.apply(check_codes, axis=1)

In [325]:
gdf_for_concat = []
new_codes_STC = ['SE00', 'SE06', 'SW00', 'SW09']
new_codes_PTC = ["PW00", "PW08", "PE00", "PE08"]

for i in new_codes_PTC:
    temp = mrt_gdf.query("CODE == 'PTC'")
    temp.loc[:, 'CODE'] = i
    gdf_for_concat.append(temp)

for i in new_codes_STC:
    temp = mrt_gdf.query("CODE == 'STC'")
    temp.loc[:, "CODE"] = i
    gdf_for_concat.append(temp)

temp = mrt_gdf.query("CODE == 'BP06'")
temp.loc[:, "CODE"] = 'BP14'
gdf_for_concat.append(temp)

temp = mrt_gdf.query("CODE == 'EW04'")
temp.loc[:, "CODE"] = "CG00"
gdf_for_concat.append(temp)

temp = mrt_gdf.query("CODE == 'CC04'")
temp.loc[:, "CODE"] = "CE00"
gdf_for_concat.append(temp)

concat_list = pd.concat(gdf_for_concat, axis=0)
# concat_list

In [326]:
mrt_gdf_lines = pd.concat([mrt_gdf, concat_list], axis=0).sort_values("CODE").reset_index(drop=True)
mrt_gdf_lines.head()

,CODE,STATION_NAME,LINE,COLOR,OPENING,TYPE,SEARCHVAL,BLK_NO,ROAD_NAME,BUILDING,ADDRESS,POSTAL,X,Y,LATITUDE,LONGITUDE,geometry,lat_lng
0,BP01,Choa Chu Kang,Bukit Panjang LRT,Grey,1999-11-06,LRT,CHOA CHU KANG LRT STATION (BP1),15,CHOA CHU KANG AVENUE 4,CHOA CHU KANG LRT STATION (BP1),15 CHOA CHU KANG AVENUE 4 CHOA CHU KANG LRT ST...,689813,18119.8205469949,40744.9184182286,1.38475541089131,103.74453769781,POINT (18119.821 40744.918),POINT (1.385 103.745)
1,BP02,South View,Bukit Panjang LRT,Grey,1999-11-06,LRT,SOUTH VIEW LRT STATION (BP2),50,CHOA CHU KANG WAY,SOUTH VIEW LRT STATION (BP2),50 CHOA CHU KANG WAY SOUTH VIEW LRT STATION (B...,688265,18203.72427563,40252.0686228592,1.38029828742399,103.745291799824,POINT (18203.724 40252.069),POINT (1.380 103.745)
2,BP03,Keat Hong,Bukit Panjang LRT,Grey,1999-11-06,LRT,KEAT HONG LRT STATION (BP3),30,CHOA CHU KANG WAY,KEAT HONG LRT STATION (BP3),30 CHOA CHU KANG WAY KEAT HONG LRT STATION (BP...,688266,18622.5893422375,40064.6246900782,1.3786032506373,103.749055668925,POINT (18622.589 40064.625),POINT (1.379 103.749)
3,BP04,Teck Whye,Bukit Panjang LRT,Grey,1999-11-06,LRT,TECK WHYE LRT STATION (BP4),10,CHOA CHU KANG WAY,TECK WHYE LRT STATION (BP4),10 CHOA CHU KANG WAY TECK WHYE LRT STATION (BP...,688267,19140.8000152688,39852.4607739244,1.37668467920808,103.753712232337,POINT (19140.800 39852.461),POINT (1.377 103.754)
4,BP05,Phoenix,Bukit Panjang LRT,Grey,1999-11-06,LRT,PHOENIX LRT STATION (BP5),40,CHOA CHU KANG ROAD,PHOENIX LRT STATION (BP5),40 CHOA CHU KANG ROAD PHOENIX LRT STATION (BP5...,689483,19617.4881601976,40065.9404090624,1.37861545104083,103.757995558346,POINT (19617.488 40065.940),POINT (1.379 103.758)


In [327]:
interchanges = mrt_gdf["STATION_NAME"].value_counts()
interchanges = interchanges[interchanges > 1].index

mrt_gdf['interchange'] = mrt_gdf["STATION_NAME"].apply(lambda x: True if x in interchanges else False)

In [328]:
# Create the 'grouper' column
mrt_gdf_lines["grouper"] = mrt_gdf_lines["CODE"].str.slice(0, 2)

# Group by 'grouper' and create LineString geometries
def create_linestring(group):
    # Check if there are at least two points to create a LineString
    if len(group) > 1:
        return LineString(group.to_list())
    else:
        # Return the original geometry if not enough points for a LineString
        return group.iloc[0]

# Apply the function to create LineString geometries
test_df = mrt_gdf_lines.groupby(["grouper", "COLOR"])["lat_lng"].apply(create_linestring)

# Convert the resulting Series to a GeoDataFrame
geo_df2 = gpd.GeoDataFrame(test_df, geometry="lat_lng").reset_index()

lines_gdf = geo_df2[geo_df2.geometry.type == "LineString"]
lines_gdf

,grouper,COLOR,lat_lng
0,BP,Grey,"LINESTRING (1.385 103.745, 1.380 103.745, 1.37..."
1,CC,Orange,"LINESTRING (1.299 103.846, 1.297 103.851, 1.29..."
2,CE,Orange,"LINESTRING (1.294 103.860, 1.282 103.859, 1.27..."
3,CG,Green,"LINESTRING (1.327 103.946, 1.335 103.962, 1.35..."
4,DT,Blue,"LINESTRING (1.379 103.762, 1.370 103.764, 1.36..."
5,EW,Green,"LINESTRING (1.373 103.949, 1.353 103.945, 1.34..."
6,NE,Purple,"LINESTRING (1.265 103.822, 1.280 103.839, 1.28..."
7,NS,Red,"LINESTRING (1.333 103.742, 1.349 103.750, 1.35..."
8,PE,Grey,"LINESTRING (1.405 103.902, 1.399 103.906, 1.39..."
10,PW,Grey,"LINESTRING (1.405 103.902, 1.410 103.905, 1.41..."


In [329]:
color_dict = {
    "Red": "#c03731",
    "Green": "#1f844d",
    "Purple": "#953aa6",
    "Orange": "#fba01d",
    "Blue": "#134f9a",
    "Brown": "#9d6633",
    "Grey": "#8c958c",
}

m_1 = folium.Map(location=[1.3521, 103.8198], zoom_start=12, tiles="cartodbpositron") # tiles="cartodbpositron"

for index, row in geo_df2.iterrows():
    color = color_dict[row['COLOR']]
    locations = [(coord[0], coord[1]) for coord in row["lat_lng"].coords]
    folium.PolyLine(
        locations=locations,
        color=color,
        weight=2,
    ).add_to(m_1)

for index, row in mrt_gdf.iterrows():
    color = color_dict[row["COLOR"]]
    tooltip = f"""
    <div style="color: white; background-color: {color}; padding: 5px; border-radius: 5px;">
        <b>{row['CODE']}: {row['STATION_NAME']} {row['TYPE']} Station</b>
    </div>
    """
    if row['interchange']:
        radius=6
        folium.CircleMarker(
            location=[row["LATITUDE"], row["LONGITUDE"]],
            radius=radius,
            weight=2,
            color="black",  # colour of corder
            fill_color=color,
            fill_opacity=1,
            tooltip=tooltip,
        ).add_to(m_1)
    else:
        radius=3
        folium.CircleMarker(
            location=[row["LATITUDE"], row["LONGITUDE"]],
            radius=radius,
            weight=1,
            color=color,
            fill_color=color,
            fill_opacity=1,
            tooltip=tooltip,
        ).add_to(m_1)


m_1

<img src='https://www.lta.gov.sg/content/dam/ltagov/getting_around/public_transport/rail_network/pdf/tel3_sm-en.png' width='100%'>